# Tropical Cyclone Return Period Calculation

This Python script calculates the return period for tropical cyclones at different locations. The return period is the average time between events of a certain intensity, such as cyclones of a specific strength or wind speed, and is used in risk assessments.

This scripts has been developed by Odériz (2025), inspired by the work of Bloemendaal et al. (2020).

## Required Libraries

Before running the script, make sure to install the following libraries:

- `pandas`: For handling and manipulating the cyclone data.
- `numpy`: For mathematical operations and array handling.
- `math`: For advanced mathematical functions.
- `matplotlib`: For plotting graphs and visualizations.
- `random`: For generating random data (if needed for simulations).

You can install these libraries using the following command:

```bash
pip install pandas numpy matplotlib random


In [ ]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
import matplotlib.pyplot as plt
import random

In [9]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)

    Parameters
    ----------
    lon1,lat1 : coordinates location 1
    lon2,lat2 : coordinates location 2

    Returns
    -------
    distance in km.

    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r


def import_storm_data(basin, pathSTORM):
    """
    Import and process storm data from multiple files.

    Parameters
    ----------
    basin : str
        The basin ID (e.g., 'ATL' for Atlantic).
    pathSTORM : str
        The path to the directory containing the storm data files.

    Returns
    -------
    list
        A list of pandas DataFrames, each containing storm data from a file.
    """

    # Define an empty list to store dataframes
    dataframes = []

    # Loop through the 12 data files (from 0 to 11)
    for i in range(12):
        # Construct the file path dynamically
        file_path = f'{pathSTORM}STORM_DATA_IBTRACS_{str(basin)}_1000_YEARS_{i}.txt'
        
        # Read the data file
        data = np.loadtxt(file_path, delimiter=',')
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Apply the incrementing logic
        incrementing_array = np.cumsum(np.r_[0, np.diff(df[0]) != 0])
        df[0] = incrementing_array

        df = df.rename(columns={0: 'season', 1: 'month', 2: 'sid',3: 'iso_time', 4: 'Basin_ID', 5: 'Latitude',
                        6: 'Longitude', 7: 'wmo_pres', 8: 'wmo_wind', 9: 'Radius', 10: 'usa_sshs', 11: 'landfall', 12: 'Dist2land'})  
        
        # Append the dataframe to the list
        dataframes.append(df)

    return dataframes


def random_dataframes(pathSTORM0,IB_Storms_Per_Year0,nSTORM,basin):
    """
    Select random storm data from multiple files and calculate the estimated number of years for storms.

    Parameters
    ----------
    pathSTORM0 : str
        The path to the directory containing the storm data files.
    IB_Storms_Per_Year0 : float
        The average number of storms per year.
    nSTORM : int
        The number of random storm samples to select.
    basin : str
        The basin ID (e.g., 'ATL' for Atlantic).

    Returns
    -------
    tuple
        A tuple containing:
        - DataFrame with random storm data.
        - Estimated number of years for storms.
    """

    dataframes_0 = import_storm_data(basin, pathSTORM0)
    random_combinations = [(random.randint(0, 11), random.randint(0, 1000)) for _ in range(nSTORM)]

    df_all_0=pd.DataFrame()

    for idx, (num1, num2) in enumerate(random_combinations, start=1):
        
        df_0 = dataframes_0[num1]
        df_0=df_0[df_0['season']==num2]
        df_all_0 = pd.concat([df_all_0, df_0], ignore_index=True)

    storm_counts_0 = df_all_0.groupby('season')['sid'].nunique().reset_index()
    total_storms_0 = storm_counts_0['sid'].sum()
    Estimated_Years_STORMS0=total_storms_0/IB_Storms_Per_Year0

    return df_all_0,Estimated_Years_STORMS0



In [ ]:
# Initialize iteration and parameters for the simulation
iteration = 0  # The iteration number (can be incremented during multiple runs)
nyear = 6000  # The number of years to simulate or consider
nperiod0 = 3  # Number of periods you want to study to select in period_names

# Define the different periods or phases of interest
period_names = [
    'IB1980-2021LANINA.v1', 
    'IB1980-2021ENSO_NEUTRAL.v1', 
    'IB1980-2021ELNINO.v1', 
    'IB1980-2021'
]

# Number of years in the IB period (1980 to 2021, inclusive)
nIB = 2021 - 1980 + 1  # This calculates the number of years from 1980 to 2021

# List of Basins for which the data needs to be processed
BASINS_0 = ['EP', 'NA', 'NI', 'SI', 'SP', 'WP']  # EP: Eastern Pacific, NA: North Atlantic, etc.

# Path to the output directory where results will be stored
pathOut = r'D:\01.Papers\2024_ENSO_cyclones\05_return_period/'  # Make sure the path is correct

# Folder name (typically used for saving the output or intermediate files)
folder_name = 'List_of_Cities.xlsx'  # Define the folder name 

In [ ]:

#==================================================== 
# Load cities                 
#==================================================== 
df=pd.read_excel(pathOut+folder_name,header=0,keep_default_na=False)
latitudes=df['LATITUDE']
longitudes=df['LONGITUDE']
basins=df['BASIN']
names=df['NAME']
capitals=df['CAPITAL CITY']

radius=111
      
returnperiods=[2]
returnperiods.extend(np.linspace(5,50,10))
returnperiods.extend(np.linspace(60,90,4))
returnperiods.extend(np.linspace(100,2000,20))

#Please check the definition of wind speed! If it's 1-minute sustained wind, use the following: 
#wind_items=[20,25,30,33,35,40,42,45,50,55,58,60,65,70,75,80,85]
#In STORM, the wind speeds are 10-min average, so the Saffir-Simpson category thresholds need to be converted from 1-min to 10-min: 
wind_items=[20,25,29,30,35,37.6,40,43.4,45,50,51.1,55,60,61.6,65,70,75]

period=period_names[nperiod0]

path0=r'C:\Users\oderizi\miniconda\scripts\STORM/'+period+'/'
pathSTORM0=path0+'tracks/'
basins_id0=np.load(path0+'BASINLIST_INTERP.npy',allow_pickle=True,encoding='latin1').item()
num_storms_IB = sum(1 for v in basins_id0.values() if v) 
IB_Storms_Per_Year0=num_storms_IB/nIB

wind_dict={i:[] for i in range(len(latitudes))}


for ibasin in range (0,6):
    basin=BASINS_0[ibasin]
    print(basin)
    data,Estimated_Years_STORMS0=random_dataframes(pathSTORM0,IB_Storms_Per_Year0,nyear,basin)
    time,lat,lon,wind=data[data.columns[3]],data[data.columns[5]],data[data.columns[6]],data[data.columns[8]]
    del data
        
    indices=[i for i,x in enumerate(time) if x==0]
    indices.append(len(time))
    i=0

    #loop over all TCs in the dataset
    while i<len(indices)-1:
        start=indices[i]
        end=indices[i+1]
        
        latslice=lat[start:end].values
        lonslice=lon[start:end].values
        windslice=wind[start:end].values
                
        for l in range(len(latitudes)): #for every city
            if basins[l]==basin:
                lat_loc=latitudes[l]
                lon_loc=longitudes[l]
                wind_loc=[]
                if lon_loc<0.:
                    lon_loc+=360        
        
                for j in range(len(latslice)):
                    #calculate the distance between the track and the capital city
                    distance=haversine(lonslice[j],latslice[j],lon_loc,lat_loc)
                    
                    if distance<=radius:
                        wind_loc.append(windslice[j])
                
                if len(wind_loc)>0.:
                    if np.max(wind_loc)>=18.:
                        wind_dict[l].append(np.max(wind_loc)) #store the maximum wind speed for the TC

        i=i+1

#################################################

#wind_dict=wind_dicts[f'wind_dict_{iterations}'] 

for i in range(len(wind_dict)):
    name=names[i]
    city=capitals[i]
    df_winds=pd.DataFrame()
    
    if len(wind_dict[i])>0.:    
        df=pd.DataFrame({'Wind': wind_dict[i]})
        df['Ranked']=df['Wind'].rank(ascending=0)
        df=df.sort_values(by=['Ranked'])
        ranklist=df['Ranked'].tolist()
        windlist=df['Wind'].tolist() 
        
        rpwindlist=[]  
        NAME_city_1=[] 
        R_period_1=[]
        Wind_velocity_1=[]
        
        NAME_city_2=[] 
        R_period_2=[]
        Wind_velocity_2=[]
            
        for m in range(len(ranklist)):
            weibull=(ranklist[m])/(len(ranklist)+1.) #weibulls plotting formula. This yields the exceendance probability per event set
            r=weibull*(len(ranklist)/nyear) #  nyear  Estimated_Years_STORMS0 multiply by the average number of events per year to reach the exceedence probability per year
            rpwindlist.append(1./r) #convert to year instead of probability

        rpwindlist=rpwindlist[::-1]         
        windlist=windlist[::-1] 
        
        for rp in returnperiods:
            windint=np.interp(rp,rpwindlist,windlist)
            NAME_city_1.append(city) 
            R_period_1.append(rp)
            Wind_velocity_1.append(windint)

        for w in wind_items:
            rp_int=np.interp(w,windlist,rpwindlist)
            NAME_city_2.append(city) 
            Wind_velocity_2.append(w)
            R_period_2.append(rp_int)


        df_winds ['Name']= NAME_city_2
        df_winds ['Wind_speed']= Wind_velocity_2
        df_winds ['RP'] = R_period_2

    df_winds.to_csv(pathOut+city+'_'+period+'_'+str(iteration)+'.csv', index=False) 

EP
NA
NI
SI
SP
WP
